# Google Earth Engine Panel Data Creation

## Initialize

In [ ]:
#!pip install geemap
##!pip install ee

In [1]:
!pip install uszipcode

You should consider upgrading via the '/home/ubuntu/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#!pip install psycopg2

In [3]:
#!pip install psycopg2-binary

In [4]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time
from enum import Enum


In [5]:
# Postgres
import psycopg2

# Zip code info
from uszipcode import SearchEngine

/home/ubuntu/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [7]:
connection = psycopg2.connect(
    user = "postgres",
    password = "
    host = "18.204.57.173",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()

In [8]:
#sr = SearchEngine()

def insert_panel_city_tf(year, city, water_area, tree_area, grass_area, turf_area, 
                        impervious_area, soil_area, total_area):

    connection.rollback()
    
    state = 'CA'
    county = 'LA County'
    
    panel_city_dict ={ 'item' : (state, 
                                    county, 
                                    city, 
                                    year, 
                                    round(total_area, 8),
                                    round(water_area, 8),
                                    round(grass_area, 8), 
                                    round(tree_area, 8),
                                    round(impervious_area, 8), 
                                    round(soil_area, 8), 
                                    round(turf_area,  8)
                                    )
    }

    
    print(panel_city_dict)
    
    columns= panel_city_dict.keys()
    
    for i in panel_city_dict.values():
        
        query = '''

        INSERT INTO panel_city_tf (state, 
                                    county, 
                                    city_neighborhood, 
                                    year, 
                                    polygon_area, 
                                    water_area, 
                                    lawn_area, 
                                    tree_area, 
                                    impervious_area, 
                                    soil_area, 
                                    turf_area 
                                    )
            VALUES {}; '''.format(i)

        try:
            cursor.execute(query)
        
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
    
        finally:
        
            if connection is not None:
                connection.commit()

In [9]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select zipcode 
from zipcode_detail
where county = 'Los Angeles County'
order by zipcode;

"""

zipcodes_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

zipcode_list = zipcodes_df['zipcode'].values.tolist()

In [10]:
sr = SearchEngine()

def insert_panel_zipcode_tf(year, zipcode, water_area, tree_area, grass_area, turf_area, 
                        impervious_area, soil_area, total_area,
                        tree_ndvi_mean, tree_ndvi_max, tree_ndvi_min,
                        grass_ndvi_mean, grass_ndvi_max, grass_ndvi_min):

    connection.rollback()
    
    z = sr.by_zipcode(zipcode)
    city = z.major_city
    state = z.state_abbr
    #print("state", state)
    county = z.county
    median_income = z.median_household_income
    
    if median_income is None:
        median_income = 0.0
    
    if tree_ndvi_mean is None:
        tree_ndvi_mean = 0.0

    if tree_ndvi_max is None:
        tree_ndvi_max = 0.0
    
    if tree_ndvi_min is None:
        tree_ndvi_min = 0.0
        
    if grass_ndvi_mean is None:
        grass_ndvi_mean = 0.0
        
    if grass_ndvi_max is None:
        grass_ndvi_max = 0.0
    
    if grass_ndvi_min is None:
        grass_ndvi_min = 0.0
    
    panel_zipcode_dict ={ 'item' : (state, 
                                    county, 
                                    zipcode, 
                                    city, 
                                    year, 
                                    round(total_area, 8),
                                    round(water_area, 8),
                                    round(grass_area, 8), 
                                    round(tree_area, 8),
                                    0.0, 
                                    round(impervious_area, 8), 
                                    round(soil_area, 8), 
                                    round(turf_area,  8),
                                    median_income, 
                                    0.0,
                                    round(tree_ndvi_mean, 8), 
                                    round(tree_ndvi_max, 8),
                                    round(tree_ndvi_min, 8),
                                    round(grass_ndvi_mean, 8),
                                    round(grass_ndvi_max, 8),
                                    round(grass_ndvi_min, 8))
    }

    
    print(panel_zipcode_dict)
    
    columns= panel_zipcode_dict.keys()
    
    for i in panel_zipcode_dict.values():
        
        query = '''

        INSERT INTO panel_zipcode_tf (state, 
                                    county, 
                                    zipcode, 
                                    city_neighborhood, 
                                    year, 
                                    polygon_area, 
                                    water_area, 
                                    lawn_area, 
                                    tree_area, 
                                    pv_area, 
                                    impervious_area, 
                                    soil_area, 
                                    turf_area, 
                                    median_income, 
                                    water_usage,
                                    tree_ndvi_mean, 
                                    tree_ndvi_max, 
                                    tree_ndvi_min,
                                    grass_ndvi_mean, 
                                    grass_ndvi_max, 
                                    grass_ndvi_min)
            VALUES {}; '''.format(i)
        

        try:
            if total_area != 0.0:
                cursor.execute(query)
        
        except (Exception, psycopg2.DatabaseError) as error:
            print("DB Insert error: ", error)
    
        finally:
        
            if connection is not None:
                connection.commit()

In [11]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [12]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load Feature Collection - Shapefiles

In [13]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sc_county = counties.filter(ee.Filter.eq('NAME', 'Santa Clara'))

In [14]:
la_county, sc_county

(<ee.featurecollection.FeatureCollection at 0x7faee60014c0>,
 <ee.featurecollection.FeatureCollection at 0x7faee60011c0>)

In [15]:
#Income Data
la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")

## Load NAIP Imagery

In [16]:
def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image
     

In [17]:
def get_images(param_dict):
    source_image_collection = param_dict['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

    
    

## Load Labeled Data

In [18]:
# ## Loading feature collections from Google Earth Engine

# water_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0610")
# water_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0701_400")
# vegetation_trees = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_torrance")
# vegetation_grass = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_torrance").limit(400)
# turf_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance1")
# turf_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance2")
# impervious_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance1").limit(35)
# impervious_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance2").limit(35)
# soil = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

# water = water_1.merge(water_2)
# turf = turf_1.merge(turf_2)
# impervious= impervious_1.merge(impervious_2)


In [19]:
# class ClassEnum(Enum):

#     '''
#     Enum class for mapping classes to new integers 
#     (previous_class_label, new_class_label)
#     '''

#     WATER = (1, 0)
#     TREES = (2, 1)
#     GRASS = (3, 2)
#     TURF = (4, 3)
#     IMPERVIOUS = (6, 4)
#     SOIL = (7, 5)

# def conditional_class(cls):
#     def map_feature(feat):
#         mapper = ClassEnum[cls].value
#         return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(mapper[0]),feat.set({'landcover': mapper[1]}),feat)
#     return map_feature

# water_tr = water.map(conditional_class('WATER'))
# trees_tr = vegetation_trees.map(conditional_class('TREES'))
# grass_tr = vegetation_grass.map(conditional_class('GRASS'))
# turf_tr = turf.map(conditional_class('TURF'))
# impervious_tr = impervious.map(conditional_class('IMPERVIOUS'))
# soil_tr = soil.map(conditional_class('SOIL'))

# LABELED_SET = water_tr.merge(trees_tr).merge(grass_tr).merge(turf_tr).merge(impervious_tr).merge(soil_tr)

In [20]:
# water_test = ee.FeatureCollection("projects/california-lawn-detection/assets/water_test")
# vegetation_trees_test = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_test")
# vegetation_grass_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_test")
# turf_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_test")
# impervious_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_reduced_test")
# soil_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

# TEST_SET = water_test.merge(vegetation_trees_test).merge(vegetation_grass_test).merge(turf_test).merge(impervious_test).merge(soil_test)

## Build Training Set

In [21]:
training_image_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020],
        'counties': {'la_county': la_county}
         }

TRAINING_IMAGE = get_images(training_image_params)['2020_la_county']

In [22]:
Map.addLayer(TRAINING_IMAGE, {}, 'TRAINING_IMAGE')

In [23]:
# Overlay the points on the imagery to get training.
LABEL = 'landcover'
BANDS = ['R', 
         'G', 
         'B', 
         'N', 
         'NDVI',
         'N_Entropy', 
         'N_Contrast', 
         'N_Gearys']

# Define classes
CLASSES = ['water',
           'vegetation_trees',
           'vegetation_grass',
           'turf',
           'impervious',
           'soil']

OLD_CLASSES = ['water_pools'] + CLASSES[1:]

# Change classes to include lakes
NEW_CLASSES = OLD_CLASSES + ['water_natural']
NEW_CLASSES[0] = 'water_pools'

# train_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
#   'collection': LABELED_SET,
#   'properties': [LABEL],
#   'scale': 1
# })

# test_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
#   'collection': TEST_SET,
#   'properties': [LABEL],
#   'scale': 1
# })

In [24]:
# print("Training Set Size in Pixels", train_data.aggregate_count('R').getInfo())

In [25]:
# print("Test Set Size in Pixels", test_data.aggregate_count('R').getInfo())

## [OLD] Machine Learning Model

In [26]:
# clf = ee.Classifier.smileRandomForest(numberOfTrees = 230, minLeafPopulation = 50, bagFraction= 0.6)\
#                    .train(train_data, LABEL, BANDS)
# clf

In [27]:
# training_image_classified = TRAINING_IMAGE.select(BANDS)\
#                                           .classify(clf)


In [28]:
# legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
# legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

# Map.addLayer(training_image_classified, {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification')

In [29]:
# training_image_classified.bandNames().getInfo()

In [30]:
# Map

## [NEW] TF Machine Learning Model

In [31]:
def get_TF_classified_image(image, bands, tf_model, classes, name='classification'):
    
    '''
    Use a TF model hosted on Google AI Platform to classify an EE image.
    '''
    
    # Select bands from training image for classification
    selected_image = image.select(bands)

    # Get the predictions
    predictions = tf_model.predictImage(selected_image.float().toArray())
    probabilities = predictions.arrayFlatten([classes])
    classified_image = predictions.arrayArgmax().arrayGet([0]).rename(name)
    
    return classified_image, predictions, probabilities

In [32]:
def get_ensembled_classified_image(image, bands, model_dicts, combined_name='combined_classification'):
    
    """
    Get the ensembled classified image by getting the max prediction probability
    for each pixel across all models. 
    
    Each dict in model_dicts should contain at a minimum the following keys for a particular model:
        'model': the ee.Model object
        'classes': the classes that the model will try to predict
        'image_name': the name of the classified image from the model 
    """
    
    output_images = []
    combined_probs = None
    
    for model_dict in model_dicts:
        # Unpack the model metadata
        model = model_dict['model']
        classes = model_dict['classes']
        image_name = model_dict['image_name']
        
        # Predict on classified image
        temp_image, temp_preds, temp_probs = get_TF_classified_image(image, bands, model, classes, name=image_name)
        
        # If no combined_probs set, set it to temp_probs
        if combined_probs is None:
            combined_probs = temp_probs
        
        # Check if classes are not the same, then add missing bands 
        cur_prob_bands = set(combined_probs.bandNames().getInfo())
        add_cur_bands = list(cur_prob_bands - set(classes))
        add_new_bands = list(set(classes) - cur_prob_bands)
        
        if add_cur_bands:
            temp_probs = temp_probs.addBands(combined_probs, add_cur_bands)
        if add_new_bands:
            combined_probs = combined_probs.addBands(temp_probs, add_new_bands)
        
        # Get the max probs across the two images
        combined_probs = combined_probs.max(temp_probs)
        output_images.append(temp_image)
    
    # Get the final combined classification image based on maximum probabilities
    classified_image = combined_probs.toArray().arrayArgmax().arrayGet([0]).rename(combined_name)
    output_images.append(classified_image)
    
    return output_images

In [33]:
# Point to the TF model(s) to be used for inference
PROJECT = 'w210-351617'
VERSION_NAME = 'v0'
OLD_MODEL_NAME = 'CNN_Nbands_model'
NEW_MODEL_NAME = 'CNN_Nbands_sep_cls_wlake_model'

input_dim = [12,12]

# Point to the old model hosted on AI Platform.
old_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=OLD_MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

# Point to the old model hosted on AI Platform.
new_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=NEW_MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

In [34]:
model_dicts = [
    {'model': old_tf_model, 'model_name': 'old_model', 'classes': OLD_CLASSES, 'image_name':'old_classification'},
    {'model': new_tf_model, 'model_name': 'new_model', 'classes': NEW_CLASSES, 'image_name':'new_classification'},
]

# Classify the training image per model + ensembled
classified_images = get_ensembled_classified_image(TRAINING_IMAGE, BANDS, model_dicts)

# Remap the combined image so lakes are classified as pools together
fromList = [0, 1, 2, 3, 4, 5, 6]
toList = [0, 1, 2, 3, 4, 5, 0]

classified_images[-1] = classified_images[-1].remap(fromList, toList)

In [35]:
old_legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']
new_legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911', '#191970']

colors_list = [old_legend_colors, new_legend_colors, old_legend_colors]

for colors, image in zip(colors_list, classified_images):
    layer_name = image.getInfo()['bands'][0]['id']
    Map.addLayer(image, {'min': 0, 'max': len(colors)-1, 'palette': colors}, layer_name)

In [36]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

## Binary Classification and Area Calculation

In [37]:
def area_calculation(image, class_number, shape, pixel_scale = 1):

    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    #area_sq_km = area_sq_m / 1e6

    return area_sq_m

In [38]:
def area_calculation_city(image, class_number, shape, pixel_scale = 1):

    if type(shape) == str:
        shape = la_cities_land.filter(ee.Filter.eq('NAME', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    #area_sq_km = area_sq_m / 1e6

    return area_sq_m

In [39]:
def ndvi_calculation(image, class_number, shape, ref_image, pixel_scale=5):
    
    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))
        
    ndvi = ref_image.normalizedDifference(['N', 'R'])
    image_clipped = image.clip(shape)
    
    NDVI_for_class = ndvi.updateMask(image_clipped.select('classification').eq(class_number))
    
    reducer = ee.Reducer.mean()\
                        .combine(ee.Reducer.max(),sharedInputs=True)\
                        .combine(ee.Reducer.min(),sharedInputs=True)
    
    
    qty = NDVI_for_class.reduceRegion(
        reducer = reducer, 
        geometry = shape, 
        scale = pixel_scale, 
        maxPixels = 1e13)
    return qty



### Create Panel Data

In [40]:
#import parcel shapes so we can clip by residential areas

la_parcel_shape_filtered = ee.FeatureCollection("projects/california-lawn-detection/assets/LA_County_Parcels_Shape")\
                             .filter(ee.Filter.eq('UseType', 'Residential'))
    
la_parcel_res = la_parcel_shape_filtered.select(ee.List(['AIN', 'SitusCity','SitusZIP','SitusFullA']), 
                                                ee.List(['AIN', 'City','ZipCode','FullAddress']))


In [41]:
#import zipcode shapes so we can clip by zipcodes

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [42]:
def run_inference_city(inference_params, model_dicts, bands):
    
    #unpack inference parameter dictionary
    inference_images = get_images(inference_params)
    residential = inference_params['residential']
    city_list = inference_params['city']
    ndvi = inference_params['ndvi']
    city_shape = inference_params['city_shape']
    residential_shape = inference_params['residential_shape']
    
    #add empty lists to data dictionary
    dictionary = {}
    
    base_keys = ['year','polygon','water_area','vegetation_trees_area', 
        'vegetation_grass_area', 'turf_area', 'impervious_area',
        'soil_area', 'total_area']
    
    ndvi_keys = ['tree_ndvi_mean', 'tree_ndvi_max','tree_ndvi_min',
       'grass_ndvi_mean', 'grass_ndvi_max','grass_ndvi_min']
    
    for i in base_keys:
        dictionary[i] = []
    if ndvi:
        for i in ndvi_keys:
            dictionary[i] = []
    
    
    #warning message about selected options
    if inference_params['residential']:
        print('CLIPPING AREA TO INCLUDE RESIDENTIAL AREAS ONLY')
    if inference_params['ndvi']:
        print('RUNNING INFERENCE INCLUDING NDVI CALCULATIONS')
    if inference_params['residential'] or inference_params['ndvi']:
        print('---------------------------------------------------------------------')
    

    #iterate through data, append to data dictionary 
    for j in list(inference_images.items()):
        image_name = j[0]
        im = j[1]
        year = image_name[:4]
        
        if residential:
            im = im.clip(residential_shape)
#             imagery = im.select(bands).classify(clf)

        # For 2012, use the old model to classify imagery
        if year == '2012':
            for model_dict in model_dicts:
                if model_dict['model_name'] == 'old_model':
                    model = model_dict['model']
                    classes = model_dict['classes']
                    imagery, _, _ = get_TF_classified_image(im, bands, model, classes)
                    break
        
        # Otherwise, use the combined ensemble model to classify imagery
        else:
            classified_images = get_ensembled_classified_image(im, bands, model_dicts)
            
            # Remap the combined image so lakes are classified as pools together
            fromList = [0, 1, 2, 3, 4, 5, 6]
            toList = [0, 1, 2, 3, 4, 5, 0]

            imagery = classified_images[-1].remap(fromList, toList)
            imagery = imagery.rename('classification')
        
        assert(imagery.bandNames().getInfo() == ['classification'])
        
        for i in city_list:


            start = time()
            polygon = la_cities_land.filter(ee.Filter.eq('NAME', i))

            dictionary['year'].append(year) 
            dictionary['polygon'].append(i)

            water_area = area_calculation_city(imagery, 0, polygon, 20)
            dictionary['water_area'].append(water_area)

            vegetation_trees_area = area_calculation_city(imagery, 1, polygon, 20)
            dictionary['vegetation_trees_area'].append(vegetation_trees_area)

            vegetation_grass_area = area_calculation_city(imagery, 2, polygon, 20)
            dictionary['vegetation_grass_area'].append(vegetation_grass_area)

            turf_area = area_calculation_city(imagery, 3, polygon, 20)
            dictionary['turf_area'].append(turf_area)

            impervious_area = area_calculation_city(imagery, 4, polygon, 20)
            dictionary['impervious_area'].append(impervious_area)

            soil_area = area_calculation_city(imagery, 5, polygon, 20)
            dictionary['soil_area'].append(soil_area)

            total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
            dictionary['total_area'].append(total_area)

            end = time()
            
            print(f'City: {i}, Year: {year} ::: completed in {end-start} seconds.')
            insert_panel_city_tf(j[0][:4], i, water_area, vegetation_trees_area, vegetation_grass_area, 
                             turf_area, impervious_area, soil_area, total_area) 
    return dictionary
              
              

In [43]:
def run_inference(inference_params, model_dicts, bands):
    
    #unpack inference parameter dictionary
    inference_images = get_images(inference_params)
    residential = inference_params['residential']
    zipcode_list = inference_params['zipcodes']
    ndvi = inference_params['ndvi']
    zipcode_shape = inference_params['zipcode_shape']
    residential_shape = inference_params['residential_shape']
    
    #add empty lists to data dictionary
    dictionary = {}
    
        
    base_keys = ['year','polygon','water_area','vegetation_trees_area', 
        'vegetation_grass_area', 'turf_area', 'impervious_area',
        'soil_area', 'total_area']
    
    ndvi_keys = ['tree_ndvi_mean', 'tree_ndvi_max','tree_ndvi_min',
       'grass_ndvi_mean', 'grass_ndvi_max','grass_ndvi_min']
    
    for i in base_keys:
        dictionary[i] = []
    if ndvi:
        for i in ndvi_keys:
            dictionary[i] = []
    
    #warning message about selected options
    if inference_params['residential']:
        print('CLIPPING AREA TO INCLUDE RESIDENTIAL AREAS ONLY')
    if inference_params['ndvi']:
        print('RUNNING INFERENCE INCLUDING NDVI CALCULATIONS')
    if inference_params['residential'] or inference_params['ndvi']:
        print('---------------------------------------------------------------------')
    
    error_zipcodes = set()

    #iterate through data, append to data dictionary 
    for j in list(inference_images.items()):
        
        image_name = j[0]
        im = j[1]
        year = image_name[:4]
        
        if residential:
            im = im.clip(residential_shape)
        #             imagery = im.select(bands).classify(clf)
        
        # For 2012, use the old model to classify imagery
        if year == '2012':
            for model_dict in model_dicts:
                if model_dict['model_name'] == 'old_model':
                    model = model_dict['model']
                    classes = model_dict['classes']
                    imagery, _, _ = get_TF_classified_image(im, bands, model, classes)
                    break
        
        # Otherwise, use the combined ensemble model to classify imagery
        else:
            classified_images = get_ensembled_classified_image(im, bands, model_dicts)
            
            # Remap the combined image so lakes are classified as pools together
            fromList = [0, 1, 2, 3, 4, 5, 6]
            toList = [0, 1, 2, 3, 4, 5, 0]
            
            imagery = classified_images[-1].remap(fromList, toList)
            imagery = imagery.rename('classification')
        
        assert(imagery.bandNames().getInfo() == ['classification'])

        for i in zipcode_list:
        
            if i in error_zipcodes:
                continue
                
            start = time()
            polygon = zipcode_shape.filter(ee.Filter.eq('ZipCode', i))

            dictionary['year'].append(year) 
            dictionary['polygon'].append(i)

            water_area=0.0
            vegetation_trees_area=0.0
            vegetation_grass_area=0.0
            turf_area=0.0
            impervious_area=0.0
            soil_area=0.0
            total_area=0.0
            
            try:
                water_area = area_calculation(imagery, 0, polygon, 20)
                dictionary['water_area'].append(water_area)

                vegetation_trees_area = area_calculation(imagery, 1, polygon, 20)
                dictionary['vegetation_trees_area'].append(vegetation_trees_area)

                vegetation_grass_area = area_calculation(imagery, 2, polygon, 20)
                dictionary['vegetation_grass_area'].append(vegetation_grass_area)

                turf_area = area_calculation(imagery, 3, polygon, 20)
                dictionary['turf_area'].append(turf_area)

                impervious_area = area_calculation(imagery, 4, polygon, 20)
                dictionary['impervious_area'].append(impervious_area)

                soil_area = area_calculation(imagery, 5, polygon, 20)
                dictionary['soil_area'].append(soil_area)

            except (Exception) as error:
                error_zipcodes.add(i)
                print("+++++++ Area calculation error +++++++", f'Zip Code: {i}', error)
                continue
            
            total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
            dictionary['total_area'].append(total_area)
            
            try:
                tree_ndvi_mean = 0.0
                tree_ndvi_max = 0.0
                tree_ndvi_min = 0.0
                grass_ndvi_mean = 0.0
                grass_ndvi_max = 0.0
                grass_ndvi_min = 0.0
                
                if total_area > 0.0 and ndvi:
                    tree_ndvi_mean, tree_ndvi_max, tree_ndvi_min = ndvi_calculation(imagery, 1, polygon, ref_image = im).getInfo().values()
                    dictionary['tree_ndvi_mean'].append(tree_ndvi_mean)
                    dictionary['tree_ndvi_max'].append(tree_ndvi_max)
                    dictionary['tree_ndvi_min'].append(tree_ndvi_min)

                    grass_ndvi_mean, grass_ndvi_max, grass_ndvi_min = ndvi_calculation(imagery, 2, polygon, ref_image = im).getInfo().values()
                    dictionary['grass_ndvi_mean'].append(grass_ndvi_mean)
                    dictionary['grass_ndvi_max'].append(grass_ndvi_max)
                    dictionary['grass_ndvi_min'].append(grass_ndvi_min)

            except (Exception) as error:
                error_zipcodes.add(i)
                print("------- NDVI calculation error -------", f'Zip Code: {i}', error)
                continue
                
            end = time()
                        
            print(f'Zip Code: {i}, Year: {year} ::: completed in {end-start} seconds. Total Area {total_area}')
            
            if total_area > 0.0:
                insert_panel_zipcode_tf(j[0][:4], i, water_area, vegetation_trees_area, vegetation_grass_area, 
                             turf_area, impervious_area, soil_area, total_area,
                            tree_ndvi_mean, tree_ndvi_max, tree_ndvi_min,
                            grass_ndvi_mean, grass_ndvi_max, grass_ndvi_min)
    return dictionary
              
              

In [46]:
inference_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
#        'years' : [2010, 2012, 2014, 2016, 2018, 2020],
        'years' : [2020],
#        'zipcodes': ['90802','90732'],
        'zipcodes': zipcode_list,
        'ndvi': True,
        'residential': False,
        'residential_shape': la_parcel_res, #don't adjust this line
        'counties': {'la_county': la_county}, #don't adjust this line
        'zipcode_shape' : la_county_income_zipcode #don't adjust
         }

In [47]:
model_dicts = [
    {'model': old_tf_model, 'model_name': 'old_model', 'classes': OLD_CLASSES, 'image_name':'old_classification'},
    {'model': new_tf_model, 'model_name': 'new_model', 'classes': NEW_CLASSES, 'image_name':'new_classification'},
]




In [48]:
dictionary = run_inference(inference_params, model_dicts, BANDS)

RUNNING INFERENCE INCLUDING NDVI CALCULATIONS
---------------------------------------------------------------------
Zip Code: 90001, Year: 2020 ::: completed in 64.71435856819153 seconds. Total Area 8878203.516091608
{'item': ('CA', 'Los Angeles County', '90001', 'Los Angeles', '2020', 8878203.51609161, 661.6786499, 225107.5977557, 1312925.75532131, 0.0, 5693478.01498244, 1636434.99727532, 9595.47210693, 35097, 0.0, 0.6937799, 0.27861752, 0.04046243, 0.66210046, 0.24558605, 0.07317073)}
Zip Code: 90002, Year: 2020 ::: completed in 54.659953355789185 seconds. Total Area 7687802.794215066
{'item': ('CA', 'Los Angeles County', '90002', 'Los Angeles', '2020', 7687802.79421507, 1323.8704834, 377340.01475675, 2207305.30141159, 0.0, 3454656.66748706, 1632943.40406554, 14233.53601074, 31258, 0.0, 0.69148936, 0.28537204, 0.04, 0.68372093, 0.24980928, 0.07368421)}
Zip Code: 90003, Year: 2020 ::: completed in 67.0363392829895 seconds. Total Area 9536761.726766065
{'item': ('CA', 'Los Angeles Coun

#### Inference - City Level

In [ ]:
#load California Cities Shapefile
cities = ee.FeatureCollection("projects/california-lawn-detection/assets/Cities2015")

# Filter for LA City
la_cities = cities.filter(ee.Filter.eq('County', 'Los Angeles'))

# Filter sea
la_cities_land = la_cities.filter(ee.Filter.notEquals('Notes', '3 nautical mile offshore'))

# list of cities in LA County
city_names = []

for i in range(len(la_cities_land.getInfo().get('features'))):
    city_names.append(la_cities_land.getInfo().get('features')[i].get('properties').get('NAME'))

In [ ]:
# for cities example - we can replace city : 'Pasadena' with city_names
inference_params_city = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
#        'years' : [2010, 2012, 2014, 2016, 2018, 2020],
        'years' : [2010],
        'city': city_names,
        'ndvi': False,
        'residential': False,
        'residential_shape': la_parcel_res, #don't adjust this line
        'counties': {'la_county': la_county}, #don't adjust this line
        'city_shape' : la_cities_land #don't adjust
         }

In [ ]:
# for cities
dictionary = run_inference_city(inference_params_city, model_dicts, BANDS)

In [ ]:
df = pd.DataFrame(dictionary)
df